In [1]:
%pip install -r ../scripts/argilla_requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset
import argilla as rg
import json
import re
from typing import List, Tuple
import json
from utils import compute_argilla_entities

# dataset used
data_path = 'MichaelAI23/English_CVs'

In [3]:
data = load_dataset(data_path)
data["train"]

Dataset({
    features: ['output', 'hobbies', 'personal', 'work_experience', 'skills', 'academia', 'education', 'overall'],
    num_rows: 1000
})

In [4]:
train_val = data["train"].train_test_split(
    test_size=100, shuffle=True, seed=42
)

In [5]:
train_data = train_val["train"]
val_data = train_val["test"]

In [6]:
api_url = "https://michaelai23-argillatest.hf.space"


In [7]:
rg.init(
    api_url=api_url,
    api_key="owner.apikey",
    workspace="admin"
)

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/argilla/client/client.py:195: UserWarning: You're connecting to Argilla Server 1.26.0 using a different client version (1.28.0).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


In [8]:
CV_text = val_data[0]["overall"]

In [9]:
CV_text

'As a Business Analyst at TechSolutions Inc., I was responsible for analyzing market trends and providing strategic recommendations to drive business growth. I started my role in January 2014 after graduating in 2012 and continued until December 2018, where I successfully implemented data-driven solutions that improved operational efficiency.\n- Published research paper on "The Impact of Data Analytics in Business Decision Making" in Journal of Business Intelligence, 2012\n- Co-authored a study on "Predictive Modeling for Customer Churn Analysis" in International Conference on Business Analytics, 2013\n- Presented findings on "Optimizing Supply Chain Management through Big Data Analysis" at the Annual Meeting of Operations Research Society, 2014\n- Contributed to a book chapter titled "Emerging Trends in Business Forecasting Techniques" published by Springer, 2015\n- Received Best Paper Award for research on "Machine Learning Applications in Marketing Strategy Development" at the Acade

In [10]:
output = val_data[0]["output"]

#### Compare predictions and the labels

In [12]:
with open(f"../textmarker_predictions_tokens_small.json", "r", encoding="utf-8") as fp:
    CV_preds = json.load(fp)["model_preds"]

In [15]:
preds = list(CV_preds.values())
preds = [pred[0]["generated_text"] for pred in preds]

In [24]:
preds[0]

"{'pe':{'s': 'Michael', 'e': ' hotel.'}, 'ed':{'s': '', 'e': ''}, 'wo':{'s': 'As', 'e': ' efficiency.'}, 'sk':{'s': 'Experienced', 'e': ' growth.'}}"

In [16]:
labels = val_data["output"]

In [17]:
json.loads(preds[0].replace("\'", "\""))

{'pe': {'s': 'Michael', 'e': ' hotel.'},
 'ed': {'s': '', 'e': ''},
 'wo': {'s': 'As', 'e': ' efficiency.'},
 'sk': {'s': 'Experienced', 'e': ' growth.'}}

In [18]:
json.loads(labels[0].replace("\'", "\"")) == json.loads(preds[0].replace("\'", "\""))

True

In [19]:
sum = 0
for i in range(len(labels)):
    if json.loads(preds[i].replace("\'", "\"")) != json.loads(labels[i].replace("\'", "\"")):
        print(val_data["overall"][i])
        print(preds[i])
        print(labels[i])
        sum += 1
        print(sum)

- Proficient in designing complex technical solutions that align with business objectives and requirements, demonstrating a deep understanding of architecture principles and best practices.
- Skilled in translating high-level business strategies into detailed technical designs, ensuring scalability, security, and performance are considered at every stage of the solution development process.
- Experienced in leading cross-functional teams and collaborating with stakeholders to drive consensus on architectural decisions, fostering a culture of innovation and continuous improvement.
- Proven ability to communicate effectively with both technical and non-technical audiences, presenting complex ideas in a clear and concise manner to facilitate decision-making and project success.
- Co-authored a research paper on "Optimizing Cloud Solutions for Scalability" published in the International Journal of Cloud Computing in 2020.
- Presented findings on "Integration Strategies for Enterprise Syste

In [17]:
sum = 0
for i in range(len(labels)):
    if preds[i] != labels[i]:
        print(val_data["overall"][i])
        print(preds[i])
        print(labels[i])
        sum += 1
        print(sum)

- Proficient in designing complex technical solutions that align with business objectives and requirements, demonstrating a deep understanding of architecture principles and best practices.
- Skilled in translating high-level business strategies into detailed technical designs, ensuring scalability, security, and performance are considered at every stage of the solution development process.
- Experienced in leading cross-functional teams and collaborating with stakeholders to drive consensus on architectural decisions, fostering a culture of innovation and continuous improvement.
- Proven ability to communicate effectively with both technical and non-technical audiences, presenting complex ideas in a clear and concise manner to facilitate decision-making and project success.
- Co-authored a research paper on "Optimizing Cloud Solutions for Scalability" published in the International Journal of Cloud Computing in 2020.
- Presented findings on "Integration Strategies for Enterprise Syste

In [20]:
texts = val_data["overall"]
labels = val_data["output"]

In [25]:
labels[27]

"{'pe': {'s': 'Penelope', 'e': ' reservations.'}, 'ed': {'s': '- Master', 'e': ', 2018'}, 'wo': {'s': '\\nAssociate', 'e': ' supervision.'}, 'sk': {'s': '- Proficient', 'e': ' precision.'}}"

In [26]:
preds_filtered = [pred for idx, pred in enumerate(preds) if pred != labels[idx]]
labels_filtered = [label for idx, label in enumerate(labels) if label != preds[idx]]
texts_filtered = [txt for idx, txt in enumerate(texts) if preds[idx] != labels[idx]]

In [27]:
labels_filtered[5]

"{'pe': {'s': 'Caleb', 'e': ' style.'}, 'ed': {'s': '', 'e': ''}, 'wo': {'s': 'Solution', 'e': ' initiatives.'}, 'sk': {'s': '- Proficient', 'e': ' timelines.'}}"

In [28]:
preds_filtered[5]

"{'pe': {'s': 'Caleb', 'e': ' style.'}, 'ed': {'s': '', 'e': ''}, 'wo': {'s': 'Solution', 'e': ' timelines.'}, 'sk': {'s': '- Proficient', 'e': ' timelines.'}}"

In [29]:
texts_filtered[5]

'- Proficient in designing complex technical solutions that align with business objectives, leveraging a deep understanding of cloud technologies and architecture patterns.\n- Skilled at translating high-level requirements into detailed design specifications, ensuring scalability, reliability, and security of the solution.\n- Experienced in collaborating with cross-functional teams to drive successful implementation and delivery of projects within tight timelines.\nCaleb Gonçalves can be reached at caleb.goncalves@email.com for any inquiries. He resides at 123 Palm Avenue, Mumbai, India and his phone number is +91-9876543210. Caleb Gonçalves is a distinguished guest at The St. Regis in Mumbai, known for his impeccable taste and style.\nSolution Architect, 2014 - Present at Tech Solutions Inc.\n- Design and implement innovative solutions for clients in various industries to address their complex business challenges.\n- Collaborate with cross-functional teams to ensure the successful del

In [ ]:
token_dataset = compute_argilla_entities(preds, labels, texts, "LLM_Textmarker_mistral_7B_02_JsonFormer")

In [ ]:
for record in token_dataset:
    rg.log(record, name="llm_textmarker_with_preds")

In [255]:
# Load the dataset
dataset = rg.load(name="llm_textmarker", workspace="admin")

In [74]:
rg.delete_records(name="llm_textmarker_with_preds")

(100, 100)